In [2]:
from sklearn.metrics import mean_absolute_error as MSE
from tensorflow.keras.models import Sequential
from cmcrameri import cm
from tqdm import tqdm
import seaborn as sns
from common import *
from models import *

# Loading our data

In [3]:
df = pd.read_csv("../data/input_data/MAIN_DATASET.csv")
price = df['NO2_price'].values.reshape(-1,1)
price_days = seperate_column_to_days(price)

# Structuring our data

In [7]:
lookbehind = 7
input_width = lookbehind*24
horizon = 24
no_hours = input_width + horizon
stride = 24
hour_in_days = int(no_hours / stride)

price_dataset = []


for i in range(len(price_days) - hour_in_days+1):
    price_dataset.append(np.concatenate((price_days[i:i+hour_in_days])))


price_dataset = np.array(price_dataset)

scaler = MinMaxScaler()
price_dataset = scaler.fit_transform(price_dataset[:,:,0])
price_dataset = price_dataset[..., np.newaxis].astype(np.float32)



n,m,k = price_dataset.shape

print(price_dataset.shape)


(2151, 48, 1)


In [8]:
print(price_dataset.shape)
testing = scaler.fit_transform(price_dataset)


(2151, 48, 1)


ValueError: Found array with dim 3. MinMaxScaler expected <= 2.

# Train test split

In [4]:
train = int(0.7*n)
valid = int(0.9*n)
X_train = price_dataset[:train, :input_width]
X_valid = price_dataset[train:valid, :input_width]
X_test = price_dataset[valid:, :input_width]

Y = np.empty((n, input_width, horizon))
for step_ahead in range(1, horizon + 1):
    Y[:,:, step_ahead - 1] = price_dataset[:,step_ahead:step_ahead + input_width, 0]

Y_train = Y[:train]
Y_valid = Y[train:valid]
Y_test = Y[valid:]

In [5]:
print(X_train.shape)

(1501, 168, 1)


## Hyperparameters

In [6]:
# Common
epochs = 25
batch_size = [100, 200, 300]
eta_list = np.logspace(-2, -4, 3)
neurons_list = [128, 64, 32]
filters = 64
kernel_size = 3
strides = 3

learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                            patience=3, 
                                                            verbose=1, 
                                                            factor=0.5, 
                                                            min_lr=0.000001)
callbacks = [learning_rate_reduction]

# Setting up the univariate SimpleRNN

In [7]:
class RNN(Sequential):
    def __init__(self, eta, neurons, horizon=24):
        super(self.__class__, self).__init__()

        # RNN architecture
        self.add(keras.layers.SimpleRNN(neurons, return_sequences=True, input_shape=[None, k]))
        self.add(keras.layers.SimpleRNN(neurons, return_sequences=True))
        self.add(keras.layers.Dense(horizon))

        # Model compile settings
        opt = keras.optimizers.Adam(learning_rate=eta)

        # Compile model
        self.compile(loss='mse', optimizer=opt)


# return_sequences=True --> (256, 168, 24)
# return_sequences=False --> (256, 24)

## Grid search

In [8]:
results_columns = ["model","batch_size", "learning_rate", "epoch", "neurons", "mse_all"]
results_columns += [f"mse{i}" for i in range(1,horizon+1)] 
results = pd.DataFrame(columns=results_columns)

x_columns = [f"x{k}" for k in range(1, input_width+1)]
t_columns = [f"t{k}" for k in range(1, horizon+1)]
t_hat_columns = [f"t_hat{k}" for k in range(1, horizon+1)]
preds_columns = columns=x_columns+t_columns+t_hat_columns
results_preds = pd.DataFrame(columns=preds_columns)

epochs_colmns = [f"epoch{i}" for i in range(epochs)]
Y_test_last = Y_test[:,-1]
best_mse = np.inf

i = 0
for eta in tqdm(eta_list):
    for batch in batch_size:
        for neurons in neurons_list:
            RNN_model = RNN(eta, neurons, horizon=horizon)

            # Training
            history = RNN_model.fit(X_train, Y_train, 
                        epochs=epochs, batch_size = batch, 
                        shuffle=False, callbacks= callbacks,  
                        validation_data=(X_valid, Y_valid), verbose=0)

            # Prediction
            Y_pred_rnn = RNN_model.predict(X_test)
            Y_pred_rnn_last = Y_pred_rnn[:,-1]     
            
            # Results
            mse_val = MSE(Y_test_last, Y_pred_rnn_last)
            mse_all = np.mean((Y_test_last - Y_pred_rnn_last) ** 2, axis=0) # The mean of each column 
            
            
            # Storing results for all models
            res1 = np.array([i, batch, eta, epochs, neurons, mse_val])
            res2 = np.array(mse_all)
            res = np.concatenate([res1,res2], axis=0)
            results.loc[i] = res
            results.to_csv(f"{REPORT_DATA}univariate_simpleRNN_results.csv")

            # Storing results from the best model
            if best_mse > mse_val:
                best_mse = mse_val
                print(f"\nNew best model with MSE = {mse_val} !\n")
                # Predictions and target for best results
                preds_res =  np.concatenate([X_test[...,0], Y_test_last, Y_pred_rnn_last],axis=1)
                best_results_preds = pd.DataFrame(preds_res, columns=preds_columns)
                best_results_preds.to_csv(f"{REPORT_DATA}best_models/univariate_simpleRNN_preds_best.csv")

                
                # Best results relative to parameters
                best_results = pd.DataFrame(columns=results_columns)
                best_results.loc[i] = res
                best_results.to_csv(f"{REPORT_DATA}best_models/univariate_simpleRNN_results_best.csv")
                
                # Model evaluation result            
                best_train_val_losses = pd.DataFrame(columns=["training_loss", "validation_loss"])
                best_train_val_losses["training_loss"] = history.history["loss"]
                best_train_val_losses["validation_loss"] = history.history["val_loss"]
                best_train_val_losses.to_csv(f"{REPORT_DATA}best_models/univariate_simpleRNN_train_val_losses_best.csv")            
                
            i += 1

  0%|          | 0/3 [00:00<?, ?it/s]2021-12-16 17:52:55.617824: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-12-16 17:52:55.617967: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-16 17:52:55.618025: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-12-16 17:52:55.618066: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-12-16 17:52:55.618105: W tensorflow/stream_exec


Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00012: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 00024: ReduceLROnPlateau reducing learning rate to 0.000625.

New best model with MSE = 0.23545351115823954 !


Epoch 00013: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00018: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 00025: ReduceLROnPlateau reducing learning rate to 0.000625.

New best model with MSE = 0.08737326971345177 !


Epoch 00012: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00018: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00025: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 00019: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00021: ReduceLROnPlateau reducing

 33%|███▎      | 1/3 [06:43<13:26, 403.12s/it]


Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00024: ReduceLROnPlateau reducing learning rate to 0.000125.

New best model with MSE = 0.07699931553145921 !


Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00015: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00019: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00022: ReduceLROnPlateau reducing learning rate to 6.25e-05.

Epoch 00013: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00017: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00022: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00025: ReduceLROnPlateau reducing learning rate to 6.25e-05.

Epoch 00015: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00019: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00022: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 000

 67%|██████▋   | 2/3 [12:42<06:17, 377.63s/it]


Epoch 00020: ReduceLROnPlateau reducing learning rate to 5e-05.

Epoch 00023: ReduceLROnPlateau reducing learning rate to 2.5e-05.


100%|██████████| 3/3 [16:53<00:00, 337.72s/it]


# Setting up the univariate ConvGRU

In [9]:
class ConvGRU(Sequential):
    def __init__(self, eta, neurons_first, neurons_second, horizon=24):
        super(self.__class__, self).__init__()

        # ConvGRU architecture
        self.add(keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding="valid", input_shape=[None, k]))
        self.add(keras.layers.GRU(neurons_first, return_sequences=True))
        self.add(keras.layers.GRU(neurons_second, return_sequences=True))
        self.add(keras.layers.Dense(horizon))

        # Model compile settings
        opt = keras.optimizers.Adam(learning_rate=eta)

        # Compile model
        self.compile(loss='mse', optimizer=opt)

## Grid search

In [10]:
results_columns = ["model","batch_size", "learning_rate", "epoch", "neurons", "mse_all"]
results_columns += [f"mse{i}" for i in range(1,horizon+1)] 
results = pd.DataFrame(columns=results_columns)

x_columns = [f"x{k}" for k in range(1, input_width+1)]
t_columns = [f"t{k}" for k in range(1, horizon+1)]
t_hat_columns = [f"t_hat{k}" for k in range(1, horizon+1)]
preds_columns = columns=x_columns+t_columns+t_hat_columns
results_preds = pd.DataFrame(columns=preds_columns)

epochs_colmns = [f"epoch{i}" for i in range(epochs)]
Y_test_last = Y_test[:,-1]
best_mse = np.inf

i = 0
for eta in tqdm(eta_list):
    for batch in batch_size:
        for neurons in neurons_list:
            ConvGRU_model = ConvGRU(eta, neurons, neurons, horizon=horizon)

            # Training
            history = ConvGRU_model.fit(X_train, Y_train[:,2::strides], 
                        epochs=epochs, batch_size = batch, 
                        shuffle=False, callbacks= callbacks,  
                        validation_data=(X_valid, Y_valid[:,2::strides]), verbose=0)

            # Prediction
            Y_pred_rnn = ConvGRU_model.predict(X_test)
            Y_pred_rnn_last = Y_pred_rnn[:,-1]     
            
            # Results
            mse_val = MSE(Y_test_last, Y_pred_rnn_last)
            mse_all = np.mean((Y_test_last - Y_pred_rnn_last) ** 2, axis=0) # The mean of each column 
                        
            # Storing results for all models
            res1 = np.array([i, batch, eta, epochs, neurons, mse_val])
            res2 = np.array(mse_all)
            res = np.concatenate([res1,res2], axis=0)
            results.loc[i] = res
            results.to_csv(f"{REPORT_DATA}univariate_convGRU_results.csv")

            # Storing results from the best model
            if best_mse > mse_val:
                best_mse = mse_val
                print(f"\nNew best model with MSE = {mse_val} !\n")
                # Predictions and target for best results
                preds_res =  np.concatenate([X_test[...,0], Y_test_last, Y_pred_rnn_last],axis=1)
                best_results_preds = pd.DataFrame(preds_res, columns=preds_columns)
                best_results_preds.to_csv(f"{REPORT_DATA}best_models/univariate_convGRU_preds_best.csv")

                
                # Best results relative to parameters
                best_results = pd.DataFrame(columns=results_columns)
                best_results.loc[i] = res
                best_results.to_csv(f"{REPORT_DATA}best_models/univariate_convGRU_results_best.csv")
                
                # Model evaluation result            
                best_train_val_losses = pd.DataFrame(columns=["training_loss", "validation_loss"])
                best_train_val_losses["training_loss"] = history.history["loss"]
                best_train_val_losses["validation_loss"] = history.history["val_loss"]
                best_train_val_losses.to_csv(f"{REPORT_DATA}best_models/univariate_convGRU_train_val_losses_best.csv")            
                
            i += 1

  0%|          | 0/3 [00:00<?, ?it/s]


Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00016: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00023: ReduceLROnPlateau reducing learning rate to 0.00125.

New best model with MSE = 0.0513961006075748 !


Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00012: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00022: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00013: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00017: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.000625.

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00014: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 00024: ReduceLROnPlateau reducing learning rate to 0.000625.

Epoch 00004: ReduceLRO

 33%|███▎      | 1/3 [04:11<08:23, 251.55s/it]


Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00017: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00023: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00012: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00017: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00020: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00023: ReduceLROnPlateau reducing learning rate to 6.25e-05.

Epoch 00015: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00020: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00023: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00012: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00016: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00019: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00022: ReduceLROnPlateau reducing learning rate to 6.25e-05.

Epoch 00025: ReduceLROnPlateau reducing learning rate to 3.

100%|██████████| 3/3 [12:30<00:00, 250.28s/it]


# Setting up the univariate LSTM

In [11]:
class LSTM(Sequential):
    def __init__(self, eta, neurons, horizon=24):
        super(self.__class__, self).__init__()
        
        # self.add(keras.layers.Bidirectional(keras.layers.LSTM(neurons, return_sequences=True)))
        # self.add(keras.layers.Bidirectional(keras.layers.LSTM(neurons, return_sequences=True)))
        
        self.add(keras.layers.LSTM(neurons, return_sequences=True, input_shape=[None, k]))
        self.add(keras.layers.LSTM(neurons, return_sequences=True))
        self.add(keras.layers.Dense(horizon))

        # Model compile settings
        opt = keras.optimizers.Adam(learning_rate=eta)

        # Compile model
        self.compile(loss='mse', optimizer=opt)

## Grid search

In [12]:
results_columns = ["model","batch_size", "learning_rate", "epoch", "neurons", "mse_all"]
results_columns += [f"mse{i}" for i in range(1,horizon+1)] 
results = pd.DataFrame(columns=results_columns)

x_columns = [f"x{k}" for k in range(1, input_width+1)]
t_columns = [f"t{k}" for k in range(1, horizon+1)]
t_hat_columns = [f"t_hat{k}" for k in range(1, horizon+1)]
preds_columns = columns=x_columns+t_columns+t_hat_columns
results_preds = pd.DataFrame(columns=preds_columns)

epochs_colmns = [f"epoch{i}" for i in range(epochs)]

Y_test_last = Y_test[:,-1]

best_mse = np.inf

i = 0
for eta in tqdm(eta_list):
    for batch in batch_size:
        for neurons in neurons_list:
            LSTM_model = LSTM(eta, neurons, horizon=horizon)

            # Training
            history = LSTM_model.fit(X_train, Y_train, 
                        epochs=epochs, batch_size = batch, 
                        shuffle=False, callbacks= callbacks,  
                        validation_data=(X_valid, Y_valid), verbose=0)

            # Prediction
            Y_pred_rnn = LSTM_model.predict(X_test)
            Y_pred_rnn_last = Y_pred_rnn[:,-1]     
            
            # Results
            mse_val = MSE(Y_test_last, Y_pred_rnn_last)
            mse_all = np.mean((Y_test_last - Y_pred_rnn_last) ** 2, axis=0) # The mean of each column 
            
            
            # Storing results for all models
            res1 = np.array([i, batch, eta, epochs, neurons, mse_val])
            res2 = np.array(mse_all)
            res = np.concatenate([res1,res2], axis=0)
            results.loc[i] = res
            results.to_csv(f"{REPORT_DATA}univariate_LSTM_results.csv")

            # Storing results from the best model
            if best_mse > mse_val:
                best_mse = mse_val
                print(f"\nNew best model with MSE = {mse_val} !\n")
                # Predictions and target for best results
                preds_res =  np.concatenate([X_test[...,0], Y_test_last, Y_pred_rnn_last],axis=1)
                best_results_preds = pd.DataFrame(preds_res, columns=preds_columns)
                best_results_preds.to_csv(f"{REPORT_DATA}best_models/univariate_LSTM_preds_best.csv")

                
                # Best results relative to parameters
                best_results = pd.DataFrame(columns=results_columns)
                best_results.loc[i] = res
                best_results.to_csv(f"{REPORT_DATA}best_models/univariate_LSTM_results_best.csv")
                
                # Model evaluation result            
                best_train_val_losses = pd.DataFrame(columns=["training_loss", "validation_loss"])
                best_train_val_losses["training_loss"] = history.history["loss"]
                best_train_val_losses["validation_loss"] = history.history["val_loss"]
                best_train_val_losses.to_csv(f"{REPORT_DATA}best_models/univariate_LSTM_train_val_losses_best.csv")            
                
            i += 1

  0%|          | 0/3 [00:00<?, ?it/s]


Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00018: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.000625.

Epoch 00024: ReduceLROnPlateau reducing learning rate to 0.0003125.

New best model with MSE = 0.10014290712914271 !


Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00017: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 00020: ReduceLROnPlateau reducing learning rate to 0.000625.

Epoch 00023: ReduceLROnPlateau reducing learning rate to 0.0003125.

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00018: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.000625.

Epoch 00024: 

 33%|███▎      | 1/3 [11:47<23:35, 707.71s/it]


Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00014: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00017: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00020: ReduceLROnPlateau reducing learning rate to 6.25e-05.

Epoch 00023: ReduceLROnPlateau reducing learning rate to 3.125e-05.

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00013: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00016: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00019: ReduceLROnPlateau reducing learning rate to 6.25e-05.

Epoch 00022: ReduceLROnPlateau reducing learning rate to 3.125e-05.

Epoch 00025: ReduceLROnPlateau reducing learning rate to 1.5625e-05.

Epoch 00011: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00017: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00020: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00023: ReduceLROnPlateau reducing learning rat

100%|██████████| 3/3 [35:29<00:00, 709.95s/it]
